In [1]:
from pytao import Tao
import os
import pandas as pd

In [2]:
# Basic model with options
MODEL = 'cu_hxr' 

OPTIONS = f'-noplot'
INIT = f'-init $LCLS_LATTICE/bmad/models/{MODEL}/tao.init {OPTIONS}'

# Make sure this exists
assert 'LCLS_LATTICE' in os.environ

In [3]:
tao = Tao(INIT)

In [5]:
t1 = pd.read_csv("magnet_polynomial_report1.csv")

In [6]:
t1

,Device,Element,Linac Z (m)*,Control System Name*,PS String,PS Config,Imin Val,Imin Uom,Imax Val,Imax Uom,Bmin Val,Bmin Uom,Bmax Val,Bmax Uom
0,BEND,BCXDLU3,665.818300,BEND:DOG:107,BCXDLU2,String,0.0,A,250.0,A,0.0141,kG-m,4.5017,kG-m
1,BEND,BCXDLD2,746.268275,BEND:DOG:242,BCXDLD2,String Main,0.0,A,250.0,A,0.0134,kG-m,4.5227,kG-m
2,BEND,BCXDLD1,745.718275,BEND:DOG:241,BCXDLD2,String,0.0,A,250.0,A,0.0132,kG-m,4.5155,kG-m
3,BEND,BCX362,3363.707649,BEND:LTUH:506,BCX362,String Main,0.0,A,250.0,A,0.0127,kG-m,4.5398,kG-m
4,BEND,BCX322,3293.237543,BEND:LTUH:301,BCX322,String Main,0.0,A,250.0,A,0.0132,kG-m,4.5095,kG-m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1353,KICK,BKRAPM3,3121.236589,KICK:BSYH:491,NaN,Individual,0.0,A,1000.0,A,0.0000,kG-m,11.2600,kG-m
1354,YCOR,YCSP2D,2963.647003,YCOR:SPD:703,NaN,Individual,-6.0,A,6.0,A,-0.1091,kG-m,0.1091,kG-m
1355,YCOR,YCSP2S,2927.766918,YCOR:SPS:587,NaN,Individual,-2.0,A,2.0,A,-0.0210,kG-m,0.0210,kG-m
1356,XCOR,XCSP9S,3004.554785,XCOR:SPS:836,NaN,Individual,-2.0,A,2.0,A,-0.0210,kG-m,0.0210,kG-m


# Simple Tao command example

In [7]:
%%tao
set var xcor[1]|meas = 0.03;
sho var xcor[1:3]

SLD  ! this resets the lattice to design!

-------------------------
Tao> set var xcor[1]|meas = 0.03;
-------------------------
Tao> sho var xcor[1:3]
  Variable                   Slave Parameters           Meas         Model        Design  Useit_opt
  xcor[1]                    XC00[BL_KICK]          3.0000E-02    0.0000E+00    0.0000E+00       F
  xcor[2]                    XC01[BL_KICK]          0.0000E+00    0.0000E+00    0.0000E+00       F
  xcor[3]                    XC02[BL_KICK]          0.0000E+00    0.0000E+00    0.0000E+00       F
  Variable                   Slave Parameters           Meas         Model        Design  Useit_opt
-------------------------
Tao> 
-------------------------
Tao> SLD  ! this resets the lattice to design!
-------------------------
Tao> 


In [8]:
hkicker_list = tao.lat_list('HKICKER::*', 'ele.name')

## For now we reply on pytao having the ORDER of the correctors (in hkicker_list) the same as in the lattice variable (in "show var xcor"). A visual check is recommended for other parameters.

In [9]:
hkicker_list[0:10]

['XC00',
 'XC01',
 'XC02',
 'XC03',
 'XC04',
 'XC05',
 'XC06',
 'XC07',
 'XC08',
 'XC09']

# Run multiple Tao commands using table from Dorian

In [10]:
def set_HKICKER_limits(t1, output_commands=False):
    
    out=""
    t2 = t1[t1['Device']=="XCOR"]            # shrink table
    t2.index = pd.RangeIndex(len(t2.index))  # reset index
    
    hkicker_list = tao.lat_list('HKICKER::*', 'ele.name')  # existing hkickers in Tao lattice
    
    for i in range(t2.shape[0]):
        
        ele_name = t2['Element'][i]
        if ele_name in hkicker_list:
            #print(hkicker_list.index(ele_name))
            
            ix_tao = hkicker_list.index(ele_name) + 1   #shifts 0 to 1
            high_lim = t2['Bmax Val'][i]*0.1  # from kG*m to T*m
            low_lim = t2['Bmin Val'][i]*0.1   # from kG*m to T*m
            
            # Run the Tao commands
            # print("Seting hkicker with variable index:", ix_tao)
            cmd = f"set var xcor[{ix_tao}]|high_lim = {high_lim};set var xcor[{ix_tao}]|low_lim = {low_lim};"
            tao.cmd(cmd)
            
            if output_commands==True:
                out += f"set var xcor[{ix_tao}]|high_lim = {high_lim};set var xcor[{ix_tao}]|low_lim = {low_lim};"
            
    
    if output_commands==True:
        return out
    else:
        return None

In [11]:
set_HKICKER_limits(t1)

#### Check if the limits have been set correctly according to the table

In [12]:
t1[t1['Element']=="XC00"]

,Device,Element,Linac Z (m)*,Control System Name*,PS String,PS Config,Imin Val,Imin Uom,Imax Val,Imax Uom,Bmin Val,Bmin Uom,Bmax Val,Bmax Uom
273,XCOR,XC00,2018.072044,XCOR:IN20:121,NaN,Individual,-4.0,A,4.0,A,-0.0015,kG-m,0.0015,kG-m


In [13]:
%%tao
sho var xcor[1]

-------------------------
Tao> sho var xcor[1]
%ele_name         = "XC00"
%attrib_name      = "BL_KICK"
%ix_attrib        = 43
%ix_var           = 55
%ix_dvar          = -1
%ix_v1            = 1
%model            =   0.00000000E+00
%base             =   0.00000000E+00
%slave(1)%uni@branch>>ele:        1@0>>10
%slave(1)%Model_value:   0.00000000E+00
%slave(1)%Base_value:    0.00000000E+00
%design           =   0.00000000E+00
%old              =   0.00000000E+00
%meas             =   3.00000000E-02
%ref              =   0.00000000E+00
%correction       =   0.00000000E+00
%high_lim         =   1.50000000E-04
%low_lim          =  -1.50000000E-04
%step             =   1.00000000E-02
%weight           =   0.00000000E+00
%delta_merit      =   0.00000000E+00
%merit_type       = "limit"
%merit            =   0.00000000E+00
%dmerit_dvar      =   0.00000000E+00
%s                =   1.96010000E-01
%ix_key_table     = -1
%exists           = T
%good_var         = T
%good_user        = F
%good_opt  

# Below is Scratch

In [10]:
os.getcwd()

'/global/u2/w/wlou1991/beam_loss_Yuantao'

In [11]:
OPTIONS = f'-noplot'
INIT = f'-init {os.getcwd()}/tao.init {OPTIONS}'

In [12]:
INIT

'-init /global/u2/w/wlou1991/beam_loss_Yuantao/tao.init -noplot'

In [13]:
tao = Tao(INIT)

In [17]:
%%tao
sho var correctors.x

-------------------------
Tao> sho var correctors.x
  Variable                   Slave Parameters           Meas         Model        Design  Useit_opt
  correctors.x[1]            XCBSYQ5[BL_KICK]       0.0000E+00   -0.0000E+00   -0.0000E+00       F
  correctors.x[2]            XCA0[BL_KICK]          0.0000E+00   -0.0000E+00   -0.0000E+00       F
  correctors.x[3]            XCVM2[BL_KICK]         0.0000E+00   -0.0000E+00   -0.0000E+00       F
  correctors.x[4]            XCVB2[BL_KICK]         0.0000E+00   -0.0000E+00   -0.0000E+00       F
  correctors.x[5]            XCVM3[BL_KICK]         0.0000E+00   -0.0000E+00   -0.0000E+00       F
  Variable                   Slave Parameters           Meas         Model        Design  Useit_opt
-------------------------
Tao> 
